In [1]:
from azure.cosmos import CosmosClient
import pandas as pd
from datetime import date

In [2]:
today = date.today().isoformat()
today

'2020-10-24'

In [3]:
# 아래 primarykey는 read-only 입니다.
config = {
    "endpoint": "https://jang.documents.azure.com:443/",
    "primarykey": "vqAEaHFa2Ov79VlOlcf8zP1TLtsUsuZqD0LUASUqu0IFZ9pU1MfdNUkepMdvipkSziB80dUmyzyqyDwsx7ypZg=="
}

client = CosmosClient(config["endpoint"], config["primarykey"])

database_name = 'MLStocking'
database = client.get_database_client(database_name)
container_name = 'daily_price'
container = database.get_container_client(container_name)

In [5]:
# SQL
q = f'''
SELECT p.Date, p.Open, p.High, p.Low, p.Close, p.Volume
FROM daily_price p
WHERE p.code = "005930" AND p.Date >= "2019-01-01" AND p.Date <= "{today}"
'''

# Get the number of items in daily_price container
items = container.query_items(
    query=q,
    enable_cross_partition_query=True)

json_list = []
for item in items:
    json_list.append(item)
# print(json_list)

keys = json_list[0].keys()
print(keys)

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])


In [6]:
testdata = pd.DataFrame(json_list)

In [7]:
testdata['Date'] = pd.to_datetime(testdata['Date'])

In [8]:
testdata.sort_values('Date', inplace=True)

In [9]:
# testdata.index = testdata.Date

In [10]:
# testdata.drop('Date', axis=1, inplace=True)

In [11]:
testdata.reset_index(drop=True, inplace=True)

### 중복제거하는 2가지 방법. groupby가 index까지 리셋해주어 약간 편하므로 채택

In [12]:
testdata[~testdata.Date.duplicated(keep='first')] # 중복제거

,Date,Open,High,Low,Close,Volume
0,2019-01-02,39400,39400,38550,38750,7847664
2,2019-01-03,38300,38550,37450,37600,12471493
4,2019-01-04,37450,37600,36850,37450,14108958
6,2019-01-07,38000,38900,37800,38750,12748997
8,2019-01-08,38000,39200,37950,38100,12756554
...,...,...,...,...,...,...
564,2020-09-23,58400,58800,57400,58600,20111398
565,2020-09-24,57700,58600,57600,57800,17564020
566,2020-09-25,57700,58200,57700,57900,11444683
567,2020-09-28,58300,58800,57900,58200,12614080


In [13]:
testdata = testdata.groupby('Date', as_index=False).first() # 중복제거

In [14]:
testdata.columns = testdata.columns.str.lower()

In [15]:
testdata['date'] = testdata['date'].dt.strftime('%Y-%m-%d')

In [16]:
testdata

,date,open,high,low,close,volume
0,2019-01-02,39400,39400,38550,38750,7847664
1,2019-01-03,38300,38550,37450,37600,12471493
2,2019-01-04,37450,37600,36850,37450,14108958
3,2019-01-07,38000,38900,37800,38750,12748997
4,2019-01-08,38000,39200,37950,38100,12756554
...,...,...,...,...,...,...
428,2020-09-23,58400,58800,57400,58600,20111398
429,2020-09-24,57700,58600,57600,57800,17564020
430,2020-09-25,57700,58200,57700,57900,11444683
431,2020-09-28,58300,58800,57900,58200,12614080


In [235]:
def read_DBdata():
    today = date.today().isoformat()
    # 아래 primarykey는 read-only 입니다.
    config = {
        "endpoint": "https://jang.documents.azure.com:443/",
        "primarykey": "vqAEaHFa2Ov79VlOlcf8zP1TLtsUsuZqD0LUASUqu0IFZ9pU1MfdNUkepMdvipkSziB80dUmyzyqyDwsx7ypZg=="
    }

    client = CosmosClient(config["endpoint"], config["primarykey"])

    database_name = 'MLStocking'
    database = client.get_database_client(database_name)
    container_name = 'daily_price'
    container = database.get_container_client(container_name)
    
    DBdata = pd.DataFrame(json_list)
    DBdata['Date'] = pd.to_datetime(DBdata['Date'])
    DBdata.sort_values('Date', inplace=True)
    DBdata = DBdata.groupby('Date', as_index=False).first() # 중복제거
    print("Today:", today)
    print("Last Data:", DBdata['Date'].dt.strftime("%Y-%m-%d").iloc[-1])
    DBdata.columns = DBdata.columns.str.lower()
    DBdata['date'] = DBdata['date'].dt.strftime('%Y%m%d') # 날짜 포맷 변경

    return DBdata

In [236]:
read_DBdata()

today: 2020-10-23
Last Data: 2020-09-29


,date,open,high,low,close,volume
0,20190102,39400,39400,38550,38750,7847664
1,20190103,38300,38550,37450,37600,12471493
2,20190104,37450,37600,36850,37450,14108958
3,20190107,38000,38900,37800,38750,12748997
4,20190108,38000,39200,37950,38100,12756554
...,...,...,...,...,...,...
428,20200923,58400,58800,57400,58600,20111398
429,20200924,57700,58600,57600,57800,17564020
430,20200925,57700,58200,57700,57900,11444683
431,20200928,58300,58800,57900,58200,12614080


In [212]:
COLUMNS_CHART_DATA = ['date', 'open', 'high', 'low', 'close', 'volume']

COLUMNS_TRAINING_DATA_V1 = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V1_RICH = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'inst_lastinst_ratio', 'frgn_lastfrgn_ratio',
    'inst_ma5_ratio', 'frgn_ma5_ratio',
    'inst_ma10_ratio', 'frgn_ma10_ratio',
    'inst_ma20_ratio', 'frgn_ma20_ratio',
    'inst_ma60_ratio', 'frgn_ma60_ratio',
    'inst_ma120_ratio', 'frgn_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V2 = [
    'per', 'pbr', 'roe',
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'market_kospi_ma5_ratio', 'market_kospi_ma20_ratio', 
    'market_kospi_ma60_ratio', 'market_kospi_ma120_ratio', 
    'bond_k3y_ma5_ratio', 'bond_k3y_ma20_ratio', 
    'bond_k3y_ma60_ratio', 'bond_k3y_ma120_ratio'
]


def preprocess(data, ver='v1'):
    windows = [5, 10, 20, 60, 120]
    for window in windows:
        data['close_ma{}'.format(window)] = \
            data['close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = \
            data['volume'].rolling(window).mean()
        data['close_ma%d_ratio' % window] = \
            (data['close'] - data['close_ma%d' % window]) \
            / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = \
            (data['volume'] - data['volume_ma%d' % window]) \
            / data['volume_ma%d' % window]
            
        if ver == 'v1.rich':
            data['inst_ma{}'.format(window)] = \
                data['close'].rolling(window).mean()
            data['frgn_ma{}'.format(window)] = \
                data['volume'].rolling(window).mean()
            data['inst_ma%d_ratio' % window] = \
                (data['close'] - data['inst_ma%d' % window]) \
                / data['inst_ma%d' % window]
            data['frgn_ma%d_ratio' % window] = \
                (data['volume'] - data['frgn_ma%d' % window]) \
                / data['frgn_ma%d' % window]

    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = \
        (data['open'][1:].values - data['close'][:-1].values) \
        / data['close'][:-1].values
    data['high_close_ratio'] = \
        (data['high'].values - data['close'].values) \
        / data['close'].values
    data['low_close_ratio'] = \
        (data['low'].values - data['close'].values) \
        / data['close'].values
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = \
        (data['close'][1:].values - data['close'][:-1].values) \
        / data['close'][:-1].values
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = \
        (data['volume'][1:].values - data['volume'][:-1].values) \
        / data['volume'][:-1] \
            .replace(to_replace=0, method='ffill') \
            .replace(to_replace=0, method='bfill').values

    if ver == 'v1.rich':
        data['inst_lastinst_ratio'] = np.zeros(len(data))
        data.loc[1:, 'inst_lastinst_ratio'] = \
            (data['inst'][1:].values - data['inst'][:-1].values) \
            / data['inst'][:-1] \
                .replace(to_replace=0, method='ffill') \
                .replace(to_replace=0, method='bfill').values
        data['frgn_lastfrgn_ratio'] = np.zeros(len(data))
        data.loc[1:, 'frgn_lastfrgn_ratio'] = \
            (data['frgn'][1:].values - data['frgn'][:-1].values) \
            / data['frgn'][:-1] \
                .replace(to_replace=0, method='ffill') \
                .replace(to_replace=0, method='bfill').values

    return data

In [237]:
def load_data(fpath, date_from, date_to, ver='v2'):
    header = None if ver == 'v1' else 0
    data = read_DBdata()
    print(data)

    if ver == 'v1':
        data.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    # 데이터 전처리
    data = preprocess(data)
    
    # 기간 필터링
#     data['date'] = data['date'].str.replace('-', '')
#     data = data[(data['date'] >= date_from) & (data['date'] <= date_to)]
#     data = data.dropna()

    # 차트 데이터 분리
    chart_data = data[COLUMNS_CHART_DATA]

    # 학습 데이터 분리
    training_data = None
    if ver == 'v1':
        training_data = data[COLUMNS_TRAINING_DATA_V1]
    elif ver == 'v1.rich':
        training_data = data[COLUMNS_TRAINING_DATA_V1_RICH]
    elif ver == 'v2':
        data.loc[:, ['per', 'pbr', 'roe']] = \
            data[['per', 'pbr', 'roe']].apply(lambda x: x / 100)
        training_data = data[COLUMNS_TRAINING_DATA_V2]
        training_data = training_data.apply(np.tanh)
    else:
        raise Exception('Invalid version.')
    print("차트데이터:", chart_data.keys(), "\n트레이닝데이터", training_data.keys(), sep=' ')
    return chart_data, training_data

In [238]:
dataS = load_data("/Users/hwarang/Desktop/python/Stockprediction/rltrader-master/data/v1/005930.csv", '20170101', '20171231', ver='v1')

today: 2020-10-23
Last Data: 2020-09-29
         date   open   high    low  close    volume
0    20190102  39400  39400  38550  38750   7847664
1    20190103  38300  38550  37450  37600  12471493
2    20190104  37450  37600  36850  37450  14108958
3    20190107  38000  38900  37800  38750  12748997
4    20190108  38000  39200  37950  38100  12756554
..        ...    ...    ...    ...    ...       ...
428  20200923  58400  58800  57400  58600  20111398
429  20200924  57700  58600  57600  57800  17564020
430  20200925  57700  58200  57700  57900  11444683
431  20200928  58300  58800  57900  58200  12614080
432  20200929  58300  59000  58200  58200  15503563

[433 rows x 6 columns]
차트데이터: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object') 
트레이닝데이터 Index(['open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
       'close_lastclose_ratio', 'volume_lastvolume_ratio', 'close_ma5_ratio',
       'volume_ma5_ratio', 'close_ma10_ratio', 'volume_ma10_ratio',
      

In [239]:
chart, training = dataS

In [240]:
chart

,date,open,high,low,close,volume
0,20190102,39400,39400,38550,38750,7847664
1,20190103,38300,38550,37450,37600,12471493
2,20190104,37450,37600,36850,37450,14108958
3,20190107,38000,38900,37800,38750,12748997
4,20190108,38000,39200,37950,38100,12756554
...,...,...,...,...,...,...
428,20200923,58400,58800,57400,58600,20111398
429,20200924,57700,58600,57600,57800,17564020
430,20200925,57700,58200,57700,57900,11444683
431,20200928,58300,58800,57900,58200,12614080


In [241]:
training

,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio,close_ma5_ratio,volume_ma5_ratio,close_ma10_ratio,volume_ma10_ratio,close_ma20_ratio,volume_ma20_ratio,close_ma60_ratio,volume_ma60_ratio,close_ma120_ratio,volume_ma120_ratio
0,0.000000,0.016774,-0.005161,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.011613,0.025266,-0.003989,-0.029677,0.589198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.003989,0.004005,-0.016021,-0.003989,0.131297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.014686,0.003871,-0.024516,0.034713,-0.096390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.019355,0.028871,-0.003937,-0.016774,0.000593,-0.000787,0.064223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.003436,0.003413,-0.020478,0.006873,-0.034516,-0.006106,0.004815,-0.015788,-0.002323,0.015070,-0.070345,0.037811,-0.001245,0.095054,-0.033390
429,-0.015358,0.013841,-0.003460,-0.013652,-0.126663,-0.013988,-0.050909,-0.026936,-0.071782,-0.000692,-0.190656,0.022074,-0.128370,0.078258,-0.154450
430,-0.001730,0.005181,-0.003454,0.001730,-0.348402,-0.007542,-0.327502,-0.023444,-0.380198,-0.001121,-0.468747,0.022336,-0.430777,0.078296,-0.446524
431,0.006908,0.010309,-0.005155,0.005181,0.102178,0.001032,-0.236108,-0.014728,-0.285792,0.000430,-0.385875,0.026244,-0.372993,0.082287,-0.387308


In [221]:
chart

,date,open,high,low,close,volume
8551,20170102,1799000,1812000,1794000,1805000,93012
8552,20170103,1814000,1831000,1801000,1824000,147153
8553,20170104,1825000,1826000,1805000,1808000,159435
8554,20170105,1803000,1803000,1777000,1778000,219349
8555,20170106,1809000,1822000,1802000,1810000,177619
...,...,...,...,...,...,...
8789,20171221,2550000,2553000,2455000,2457000,312486
8790,20171222,2470000,2498000,2462000,2485000,223993
8791,20171226,2488000,2505000,2410000,2410000,320797
8792,20171227,2448000,2478000,2423000,2468000,214872


In [242]:
training

,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio,close_ma5_ratio,volume_ma5_ratio,close_ma10_ratio,volume_ma10_ratio,close_ma20_ratio,volume_ma20_ratio,close_ma60_ratio,volume_ma60_ratio,close_ma120_ratio,volume_ma120_ratio
0,0.000000,0.016774,-0.005161,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.011613,0.025266,-0.003989,-0.029677,0.589198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.003989,0.004005,-0.016021,-0.003989,0.131297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.014686,0.003871,-0.024516,0.034713,-0.096390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.019355,0.028871,-0.003937,-0.016774,0.000593,-0.000787,0.064223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.003436,0.003413,-0.020478,0.006873,-0.034516,-0.006106,0.004815,-0.015788,-0.002323,0.015070,-0.070345,0.037811,-0.001245,0.095054,-0.033390
429,-0.015358,0.013841,-0.003460,-0.013652,-0.126663,-0.013988,-0.050909,-0.026936,-0.071782,-0.000692,-0.190656,0.022074,-0.128370,0.078258,-0.154450
430,-0.001730,0.005181,-0.003454,0.001730,-0.348402,-0.007542,-0.327502,-0.023444,-0.380198,-0.001121,-0.468747,0.022336,-0.430777,0.078296,-0.446524
431,0.006908,0.010309,-0.005155,0.005181,0.102178,0.001032,-0.236108,-0.014728,-0.285792,0.000430,-0.385875,0.026244,-0.372993,0.082287,-0.387308


In [3]:
import platform
import locale

In [4]:
platform.system()

'Windows'